In [2]:
import pandas as pd

In [3]:
#Loaded CSV data into a dataframe objects
appointment_df = pd.read_csv('C:\\Users\\lalas\\JupyterProjects\\upwork\\appointment.csv')
task_df =pd.read_csv('C:\\Users\\lalas\\JupyterProjects\\upwork\\task.csv')
task_status_df =pd.read_csv('C:\\Users\\lalas\\JupyterProjects\\upwork\\task_status.csv')
task_type_df=pd.read_csv('C:\\Users\\lalas\\JupyterProjects\\upwork\\task_type.csv')
appt_status_df=pd.read_csv('C:\\Users\\lalas\\JupyterProjects\\upwork\\appt_status.csv')
bdc_df=pd.read_csv('C:\\Users\\lalas\\JupyterProjects\\upwork\\bdc.csv')

#combine appointment & task dataframe
combine_df = pd.merge(appointment_df, task_df, left_on='id', right_on='apptId', how='left')

In [4]:
#populate metadata information based on identifiers
combine_df['taskStatus'] = combine_df['statusId_y'].map(task_status_df.set_index('id')['status_name'])
combine_df['appointmentStatus'] = combine_df['statusId_x'].map(appt_status_df.set_index('id')['status_name'])
combine_df['taskType'] = combine_df['taskTypeId'].map(task_type_df.set_index('id')['task_type_name'])


In [5]:
print(f'Total No. of Unique appointment >> ' + str("{:,}".format(appointment_df["id"].nunique())))

Total No. of Unique appointment >> 1,125,000


In [6]:
#Populate TaskCount, UniqueTaskCount, CompletedTaskCount based on appointment Id
combine_df['TaskCount'] = combine_df.groupby(['id_x'])['id_y'].transform('size')
combine_df['TaskFollowUpCount'] = combine_df.groupby(['id_x','taskTypeId'])['id_y'].transform('size')
combine_df['UniqueTaskCount'] = combine_df.groupby(['id_x'])['taskTypeId'].transform('nunique') 
combine_df['CompletedTaskCount'] = combine_df.groupby('id_x')['statusId_y'].transform('sum')
combine_df['InCompletedTaskCount'] = combine_df.groupby(['id_x'])['id_y'].transform('size') - combine_df.groupby('id_x')['statusId_y'].transform('sum')

#clean up and data hydration
combine_df.loc[combine_df['TaskFollowUpCount'].isna(), 'TaskCount'] = 0
combine_df.loc[combine_df['TaskFollowUpCount'].isna(), 'InCompletedTaskCount'] = 0
combine_df.loc[combine_df['TaskFollowUpCount'].isna(), 'TaskFollowUpCount'] = 0
combine_df.loc[combine_df['taskStatus'].isna(), 'taskStatus'] = 'Not Available'
combine_df.loc[combine_df['appointmentStatus'].isna(), 'appointmentStatus'] = 'Not Available'
combine_df.loc[combine_df['taskType'].isna(), 'taskType'] = 'Not Available'

#Date bucket
combine_df.loc[combine_df['leadTime']<=3, 'DateBucket'] = '0-3 Days'
combine_df.loc[((combine_df['leadTime']<=10) &  (combine_df['leadTime']>=4)), 'DateBucket'] = '4-10 Days'
combine_df.loc[combine_df['leadTime']>10, 'DateBucket'] = 'More than 10 Days'


#combine_df.head(20)

In [7]:
#Load the Data frame into the a file for further processing and visualization
#combine_df.to_csv('cleanedData_new.csv', index=False) 

In [8]:
#Count of appointment not show & show
show_no_show_df = combine_df.groupby(['appointmentStatus'])['id_x'].nunique().reset_index(name='Counts')
show_no_show_df['Percentage'] = round((show_no_show_df['Counts'] / show_no_show_df['Counts'].sum())*100,2)
show_no_show_df

,appointmentStatus,Counts,Percentage
0,No Show,255768,22.73
1,Show,869232,77.27


In [9]:
#Count of unique appointment based on Date buckets
show_no_show_df = combine_df.groupby(['DateBucket'])['id_x'].nunique().reset_index(name='Counts')
show_no_show_df['Percentage'] = round((show_no_show_df['Counts'] / show_no_show_df['Counts'].sum())*100,2)
show_no_show_df

,DateBucket,Counts,Percentage
0,0-3 Days,293761,26.11
1,4-10 Days,680938,60.53
2,More than 10 Days,150301,13.36


In [10]:
#Distribution of unique appointment based on status and date buckets
show_no_show_df = combine_df.groupby(['DateBucket', 'appointmentStatus'])['id_x'].nunique().reset_index(name='Counts')
show_no_show_df['Percentage'] = round((show_no_show_df['Counts'] / show_no_show_df['Counts'].sum())*100,2)
show_no_show_df

,DateBucket,appointmentStatus,Counts,Percentage
0,0-3 Days,No Show,29969,2.66
1,0-3 Days,Show,263792,23.45
2,4-10 Days,No Show,159825,14.21
3,4-10 Days,Show,521113,46.32
4,More than 10 Days,No Show,65974,5.86
5,More than 10 Days,Show,84327,7.50


In [11]:
#Distribution of Unique appoints based on Date bucket, Task count and status
value_df = combine_df.groupby(['DateBucket','TaskCount','appointmentStatus'])['id_x'].nunique().reset_index(name='Counts')
value_df['Percentage'] = round((value_df['Counts'] / value_df['Counts'].sum())*100,2)
value_df['Total'] = value_df['Counts'].sum()
value_df.head(50)

,DateBucket,TaskCount,appointmentStatus,Counts,Percentage,Total
0,0-3 Days,0,No Show,29969,2.66,1125000
1,0-3 Days,0,Show,263792,23.45,1125000
2,4-10 Days,1,No Show,31901,2.84,1125000
3,4-10 Days,1,Show,104435,9.28,1125000
4,4-10 Days,2,No Show,112228,9.98,1125000
5,4-10 Days,2,Show,364437,32.39,1125000
6,4-10 Days,3,No Show,15696,1.40,1125000
7,4-10 Days,3,Show,52241,4.64,1125000
8,More than 10 Days,1,No Show,13287,1.18,1125000
9,More than 10 Days,1,Show,16926,1.50,1125000


In [12]:
#Distribution of Unique appointments for details analysis and how much tasks are completed and its distribution
#Recommendations are given based on detail analysis
value_df = combine_df.groupby(['DateBucket','TaskCount','appointmentStatus','InCompletedTaskCount','CompletedTaskCount'])['id_x'].nunique().reset_index(name='Counts')
#value_df['Counts'].sum()
value_df['Percentage'] = round((value_df['Counts'] / value_df['Counts'].sum())*100,2)
value_df['Total'] = value_df['Counts'].sum()
value_df.head(50)

,DateBucket,TaskCount,appointmentStatus,InCompletedTaskCount,CompletedTaskCount,Counts,Percentage,Total
0,0-3 Days,0,No Show,0.0,0.0,29969,2.66,1125000
1,0-3 Days,0,Show,0.0,0.0,263792,23.45,1125000
2,4-10 Days,1,No Show,0.0,1.0,25161,2.24,1125000
3,4-10 Days,1,No Show,1.0,0.0,6740,0.60,1125000
4,4-10 Days,1,Show,0.0,1.0,84089,7.47,1125000
5,4-10 Days,1,Show,1.0,0.0,20346,1.81,1125000
6,4-10 Days,2,No Show,0.0,2.0,55342,4.92,1125000
7,4-10 Days,2,No Show,1.0,1.0,32680,2.90,1125000
8,4-10 Days,2,No Show,2.0,0.0,24206,2.15,1125000
9,4-10 Days,2,Show,0.0,2.0,183058,16.27,1125000


In [13]:
value_df = combine_df.query('DateBucket == "More than 10 Days" and TaskCount >1'). groupby(['DateBucket','TaskCount','appointmentStatus','InCompletedTaskCount','CompletedTaskCount'])['id_x'].nunique().reset_index(name='Counts')
#value_df['Counts'].sum()
value_df['Percentage'] = round((value_df['Counts'] / value_df['Counts'].sum())*100,2)
value_df['Total'] = value_df['Counts'].sum()
value_df.head(50)

,DateBucket,TaskCount,appointmentStatus,InCompletedTaskCount,CompletedTaskCount,Counts,Percentage,Total
0,More than 10 Days,2,No Show,0.0,2.0,22538,18.77,120088
1,More than 10 Days,2,No Show,1.0,1.0,13240,11.03,120088
2,More than 10 Days,2,No Show,2.0,0.0,10334,8.61,120088
3,More than 10 Days,2,Show,0.0,2.0,30348,25.27,120088
4,More than 10 Days,2,Show,1.0,1.0,18210,15.16,120088
5,More than 10 Days,2,Show,2.0,0.0,10522,8.76,120088
6,More than 10 Days,3,No Show,0.0,3.0,3212,2.67,120088
7,More than 10 Days,3,No Show,1.0,2.0,1851,1.54,120088
8,More than 10 Days,3,No Show,3.0,0.0,1512,1.26,120088
9,More than 10 Days,3,Show,0.0,3.0,4250,3.54,120088


In [14]:
## Noshow count when all the remainders are not completed
value_df = combine_df.query('InCompletedTaskCount == 0 and appointmentStatus == "No Show"' ).groupby(['DateBucket','TaskCount','appointmentStatus','InCompletedTaskCount','CompletedTaskCount'])['id_x'].nunique().reset_index(name='Counts')
#value_df['Counts'].sum()
value_df['Percentage'] = round((value_df['Counts'] / value_df['Counts'].sum())*100,2)
value_df['Total'] = value_df['Counts'].sum()
value_df.head(50)

,DateBucket,TaskCount,appointmentStatus,InCompletedTaskCount,CompletedTaskCount,Counts,Percentage,Total
0,0-3 Days,0,No Show,0.0,0.0,29969,19.41,154392
1,4-10 Days,1,No Show,0.0,1.0,25161,16.30,154392
2,4-10 Days,2,No Show,0.0,2.0,55342,35.85,154392
3,4-10 Days,3,No Show,0.0,3.0,7791,5.05,154392
4,More than 10 Days,1,No Show,0.0,1.0,10379,6.72,154392
5,More than 10 Days,2,No Show,0.0,2.0,22538,14.60,154392
6,More than 10 Days,3,No Show,0.0,3.0,3212,2.08,154392


In [15]:
value_df = combine_df.query('InCompletedTaskCount == 0 and appointmentStatus == "No Show"' ).groupby(['DateBucket','TaskCount','appointmentStatus','InCompletedTaskCount','CompletedTaskCount'])['id_x'].nunique().reset_index(name='Counts')
#value_df['Counts'].sum()
value_df['Percentage'] = round((value_df['Counts'] / value_df['Counts'].sum())*100,2)
value_df['Total'] = value_df['Counts'].sum()
value_df.head(50)

,DateBucket,TaskCount,appointmentStatus,InCompletedTaskCount,CompletedTaskCount,Counts,Percentage,Total
0,0-3 Days,0,No Show,0.0,0.0,29969,19.41,154392
1,4-10 Days,1,No Show,0.0,1.0,25161,16.30,154392
2,4-10 Days,2,No Show,0.0,2.0,55342,35.85,154392
3,4-10 Days,3,No Show,0.0,3.0,7791,5.05,154392
4,More than 10 Days,1,No Show,0.0,1.0,10379,6.72,154392
5,More than 10 Days,2,No Show,0.0,2.0,22538,14.60,154392
6,More than 10 Days,3,No Show,0.0,3.0,3212,2.08,154392


In [16]:
#Distribution of Unique appointments for details analysis and how much tasks are completed and its distribution
#Recommendations are given based on detail analysis
value_df = combine_df.groupby(['appointmentStatus','taskType'])['id_x'].nunique().reset_index(name='Counts')
#value_df['Counts'].sum()
value_df['Percentage'] = round((value_df['Counts'] / value_df['Counts'].sum())*100,2)
value_df['Total'] = value_df['Counts'].sum()
value_df.head(50)

,appointmentStatus,taskType,Counts,Percentage,Total
0,No Show,Email,118864,7.40,1605209
1,No Show,Not Available,29969,1.87,1605209
2,No Show,Phone Call,118658,7.39,1605209
3,No Show,SMS,118478,7.38,1605209
4,Show,Email,318089,19.82,1605209
5,Show,Not Available,263792,16.43,1605209
6,Show,Phone Call,318745,19.86,1605209
7,Show,SMS,318614,19.85,1605209


In [17]:
combine_df

,id_x,storeId_x,bdcId_x,apptDate,bookingDate,statusId_x,leadTime,id_y,apptId,storeId_y,...,statusId_y,taskStatus,appointmentStatus,taskType,TaskCount,TaskFollowUpCount,UniqueTaskCount,CompletedTaskCount,InCompletedTaskCount,DateBucket
0,0,222,5,2023-08-29,2023-08-20,1,9,0.0,0.0,222.0,...,1.0,Complete,Show,Phone Call,3,1.0,2,3.0,0.0,4-10 Days
1,0,222,5,2023-08-29,2023-08-20,1,9,1.0,0.0,222.0,...,1.0,Complete,Show,Email,3,2.0,2,3.0,0.0,4-10 Days
2,0,222,5,2023-08-29,2023-08-20,1,9,2.0,0.0,222.0,...,1.0,Complete,Show,Email,3,2.0,2,3.0,0.0,4-10 Days
3,1,32,3,2023-08-29,2023-08-17,0,12,3.0,1.0,32.0,...,1.0,Complete,No Show,SMS,2,1.0,2,2.0,0.0,More than 10 Days
4,1,32,3,2023-08-29,2023-08-17,0,12,4.0,1.0,32.0,...,1.0,Complete,No Show,Email,2,1.0,2,2.0,0.0,More than 10 Days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872518,1124997,225,1,2034-08-16,2034-08-09,1,7,1578757.0,1124997.0,225.0,...,1.0,Complete,Show,SMS,1,1.0,1,1.0,0.0,4-10 Days
1872519,1124998,144,4,2034-08-16,2034-08-12,1,4,1578758.0,1124998.0,144.0,...,1.0,Complete,Show,Phone Call,2,2.0,1,2.0,0.0,4-10 Days
1872520,1124998,144,4,2034-08-16,2034-08-12,1,4,1578759.0,1124998.0,144.0,...,1.0,Complete,Show,Phone Call,2,2.0,1,2.0,0.0,4-10 Days
1872521,1124999,22,1,2034-08-16,2034-08-09,1,7,1578760.0,1124999.0,22.0,...,1.0,Complete,Show,SMS,2,2.0,1,2.0,0.0,4-10 Days


In [22]:
phone_call =combine_df.query('taskType == "Phone Call"')#.groupby(['taskType','TaskCount','appointmentStatus','InCompletedTaskCount','CompletedTaskCount'])['id_x'].nunique().reset_index(name='Counts')
phone_call.head(50)

,id_x,storeId_x,bdcId_x,apptDate,bookingDate,statusId_x,leadTime,id_y,apptId,storeId_y,...,statusId_y,taskStatus,appointmentStatus,taskType,TaskCount,TaskFollowUpCount,UniqueTaskCount,CompletedTaskCount,InCompletedTaskCount,DateBucket
0,0,222,5,2023-08-29,2023-08-20,1,9,0.0,0.0,222.0,...,1.0,Complete,Show,Phone Call,3,1.0,2,3.0,0.0,4-10 Days
11,6,183,4,2023-08-29,2023-08-24,1,5,9.0,6.0,183.0,...,1.0,Complete,Show,Phone Call,2,1.0,2,2.0,0.0,4-10 Days
22,14,101,4,2023-08-29,2023-08-25,1,4,17.0,14.0,101.0,...,1.0,Complete,Show,Phone Call,1,1.0,1,1.0,0.0,4-10 Days
23,15,168,3,2023-08-29,2023-08-14,0,15,18.0,15.0,168.0,...,1.0,Complete,No Show,Phone Call,1,1.0,1,1.0,0.0,More than 10 Days
28,18,229,1,2023-08-29,2023-08-24,1,5,23.0,18.0,229.0,...,0.0,Incomplete,Show,Phone Call,3,3.0,1,0.0,3.0,4-10 Days
29,18,229,1,2023-08-29,2023-08-24,1,5,24.0,18.0,229.0,...,0.0,Incomplete,Show,Phone Call,3,3.0,1,0.0,3.0,4-10 Days
30,18,229,1,2023-08-29,2023-08-24,1,5,25.0,18.0,229.0,...,0.0,Incomplete,Show,Phone Call,3,3.0,1,0.0,3.0,4-10 Days
38,24,3,3,2023-08-29,2023-08-20,0,9,31.0,24.0,3.0,...,1.0,Complete,No Show,Phone Call,2,2.0,1,1.0,1.0,4-10 Days
39,24,3,3,2023-08-29,2023-08-20,0,9,32.0,24.0,3.0,...,0.0,Incomplete,No Show,Phone Call,2,2.0,1,1.0,1.0,4-10 Days
43,26,97,4,2023-08-29,2023-08-21,0,8,36.0,26.0,97.0,...,1.0,Complete,No Show,Phone Call,2,1.0,2,2.0,0.0,4-10 Days


In [23]:
combine_df.query('TaskCount ==1')

,id_x,storeId_x,bdcId_x,apptDate,bookingDate,statusId_x,leadTime,id_y,apptId,storeId_y,...,statusId_y,taskStatus,appointmentStatus,taskType,TaskCount,TaskFollowUpCount,UniqueTaskCount,CompletedTaskCount,InCompletedTaskCount,DateBucket
7,4,83,2,2023-08-29,2023-08-16,0,13,5.0,4.0,83.0,...,0.0,Incomplete,No Show,SMS,1,1.0,1,0.0,1.0,More than 10 Days
12,7,37,3,2023-08-29,2023-08-24,1,5,10.0,7.0,37.0,...,1.0,Complete,Show,SMS,1,1.0,1,1.0,0.0,4-10 Days
17,11,96,2,2023-08-29,2023-08-25,1,4,13.0,11.0,96.0,...,1.0,Complete,Show,Email,1,1.0,1,1.0,0.0,4-10 Days
22,14,101,4,2023-08-29,2023-08-25,1,4,17.0,14.0,101.0,...,1.0,Complete,Show,Phone Call,1,1.0,1,1.0,0.0,4-10 Days
23,15,168,3,2023-08-29,2023-08-14,0,15,18.0,15.0,168.0,...,1.0,Complete,No Show,Phone Call,1,1.0,1,1.0,0.0,More than 10 Days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872376,1124918,266,5,2034-08-16,2034-08-11,1,5,1578635.0,1124918.0,266.0,...,1.0,Complete,Show,SMS,1,1.0,1,1.0,0.0,4-10 Days
1872380,1124922,233,2,2034-08-16,2034-08-12,1,4,1578636.0,1124922.0,233.0,...,1.0,Complete,Show,Phone Call,1,1.0,1,1.0,0.0,4-10 Days
1872391,1124928,100,1,2034-08-16,2034-08-10,1,6,1578646.0,1124928.0,100.0,...,1.0,Complete,Show,SMS,1,1.0,1,1.0,0.0,4-10 Days
1872505,1124991,231,2,2034-08-16,2034-08-11,1,5,1578744.0,1124991.0,231.0,...,0.0,Incomplete,Show,Phone Call,1,1.0,1,0.0,1.0,4-10 Days


In [30]:
value_df=combine_df.query('TaskCount ==1 ').groupby(['taskType','taskStatus'])['id_x'].nunique().reset_index()
value_df['Total'] = value_df['id_x'].sum()
value_df

,taskType,taskStatus,id_x,Total
0,Email,Complete,44511,166549
1,Email,Incomplete,10981,166549
2,Phone Call,Complete,44693,166549
3,Phone Call,Incomplete,11075,166549
4,SMS,Complete,44194,166549
5,SMS,Incomplete,11095,166549


In [33]:
value_df=combine_df.query('TaskCount ==1 ').groupby(['taskType'])['id_x'].nunique().reset_index()
value_df['Total'] = value_df['id_x'].sum()
value_df

,taskType,id_x,Total
0,Email,55492,166549
1,Phone Call,55768,166549
2,SMS,55289,166549


In [32]:
value_df=combine_df.query('TaskCount ==2 ').groupby(['taskType','taskStatus'])['id_x'].nunique().reset_index()
value_df['Total'] = value_df['id_x'].sum()
value_df

,taskType,taskStatus,id_x,Total
0,Email,Complete,219786,1028023
1,Email,Incomplete,122761,1028023
2,Phone Call,Complete,219945,1028023
3,Phone Call,Incomplete,122871,1028023
4,SMS,Complete,219773,1028023
5,SMS,Incomplete,122887,1028023


In [25]:
value_df=combine_df.query('TaskCount ==2').groupby('taskType')['id_x'].nunique().reset_index()
value_df['Total'] = value_df['id_x'].sum()
value_df

,taskType,id_x,Total
0,Email,323277,970207
1,Phone Call,323429,970207
2,SMS,323501,970207


In [35]:
value_df=combine_df.query('TaskCount ==3').groupby(['taskType','taskStatus'])['id_x'].nunique().reset_index()
value_df['Total'] = value_df['id_x'].sum()
value_df

,taskType,taskStatus,id_x,Total
0,Email,Complete,42842,188427
1,Email,Incomplete,19940,188427
2,Phone Call,Complete,42918,188427
3,Phone Call,Incomplete,19871,188427
4,SMS,Complete,42803,188427
5,SMS,Incomplete,20053,188427


In [36]:
value_df=combine_df.query('TaskCount ==3').groupby(['taskType'])['id_x'].nunique().reset_index()
value_df['Total'] = value_df['id_x'].sum()
value_df

,taskType,id_x,Total
0,Email,58184,174692
1,Phone Call,58206,174692
2,SMS,58302,174692
